In [1]:
# Kernal PCA

import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE

In [14]:

def pca(x_train, x_test, n):
    from sklearn.decomposition import PCA
    pca = PCA(n_components=2)
    xTrain=pca.fit_transform(x_train)
    xTest=pca.transform(x_test)
    explained_variance = pca.explained_variance_ratio_
    return explained_variance, x_train, x_test

def Split_Scaler(indep,dep):
    x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=0.30,random_state=0)
    scX=StandardScaler()
    x_train=scX.fit_transform(x_train)
    x_test=scX.transform(x_test)
    return x_train,x_test,y_train,y_test, scX

def cm_predition(classifier, x_test):
    y_pred=classifier.predict(x_test)

    #making the Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm=confusion_matrix(y_test,y_pred)

    # making Classification Report
    from sklearn.metrics import classification_report
    clf_report=classification_report(y_test,y_pred)

    # finding the accuracy score
    from sklearn.metrics import accuracy_score
    accuracy_score=accuracy_score(y_test,y_pred)
    
    return classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred

def logistic(x_train,y_train,x_test):
    from sklearn.linear_model import LogisticRegression
    classifier=LogisticRegression(random_state=0)
    classifier.fit(x_train,y_train)
    classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = cm_predition(classifier,x_test)
    return classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred 
    
def svm_linear(x_train,y_train,x_test):
    from sklearn.svm import SVC
    classifier=SVC(kernel='linear',random_state=0)
    classifier.fit(x_train,y_train)
    classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = cm_predition(classifier,x_test)
    return classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred 
    
def svm_nonlinear(x_train,y_train,x_test):
    from sklearn.svm import SVC
    classifier=SVC(kernel='rbf', random_state=0)
    classifier.fit(x_train,y_train)
    classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = cm_predition(classifier,x_test)
    return classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred 
    
def random(x_train,y_train,x_test):
    from sklearn.ensemble import RandomForestClassifier
    classifier=RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
    classifier.fit(x_train,y_train)
    classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = cm_predition(classifier,x_test)
    return classifier,cm,clf_report,accuracy_score,x_test,y_test ,y_pred 

def Decision(x_train,y_train,x_test):
    from sklearn.tree import DecisionTreeClassifier
    classifier=DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
    classifier.fit(x_train,y_train)
    classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = cm_predition(classifier,x_test)
    return classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred 

def knn(x_train,y_train,x_test):
    from sklearn.neighbors import KNeighborsClassifier
    classifier=KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
    classifier.fit(x_train,y_train)
    classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = cm_predition(classifier,x_test)
    return classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred 

def navie(x_train,y_train,x_test):
    from sklearn.naive_bayes import GaussianNB
    classifier=GaussianNB()
    classifier.fit(x_train,y_train)
    classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = cm_predition(classifier,x_test)
    return classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred 

# Displaying the values in the tabular frame

def pca_classification(acc_logistic, acc_svmL, acc_svmNL, acc_knn, acc_rf, acc_des, acc_nav):
    
    table=pd.DataFrame(index=["PCA"],columns=["Logistic","SVM_linear","SVM_Nonlinear","KNN","Random","Decision","Navie"])
    for number,idex in enumerate(table.index):
         table['Logistic'][idex]= acc_logistic[number]
         table['SVM_linear'][idex]= acc_svmL[number]
         table['SVM_Nonlinear'][idex]= acc_svmNL[number]
         table['KNN'][idex]= acc_knn[number]
         table['Random'][idex]= acc_rf[number]
         table['Decision'][idex]= acc_des[number]
         table['Navie'][idex]= acc_nav[number]
         
    return table




In [15]:
dataset=pd.read_csv("Social_Network_Ads.csv",index_col=None)
df1=dataset  

#Convert categorical variable into dummy/indicator variables.
df1=pd.get_dummies(df1,drop_first=True,dtype=int)
df1

# separation of independent and dependent variables
indep=df1.drop("Purchased",axis=1)
dep=df1["Purchased"]

# separate the train and test data adn apply StandardScalar techniques to standardize feaures...
x_train,x_test,y_train,y_test,scX= Split_Scaler(indep, dep)


# Applying PCA...
explained_variance_ratio, x_train, x_test = pca(x_train, x_test, 2)

acc_logistic=[]
acc_svmL=[]
acc_svmNL=[]
acc_knn=[]
acc_nav=[]
acc_des=[]
acc_rf=[]



In [16]:
# create, train and prdict the models using diff algorithms...
classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = logistic(x_train,y_train,x_test)
acc_logistic.append(accuracy_score)

classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = svm_linear(x_train,y_train,x_test)
acc_svmL.append(accuracy_score)

classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = svm_nonlinear(x_train,y_train,x_test)
acc_svmNL.append(accuracy_score)

classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = knn(x_train,y_train,x_test)
acc_knn.append(accuracy_score)

classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred= navie(x_train,y_train,x_test)
acc_nav.append(accuracy_score)

classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = Decision(x_train,y_train,x_test)
acc_des.append(accuracy_score)

classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = random(x_train,y_train,x_test)
acc_rf.append(accuracy_score)

result= pca_classification(acc_logistic,acc_svmL,acc_svmNL,acc_knn,acc_rf,acc_des,acc_nav)



/var/folders/1s/70xy4dvs6mn0zqfz6nvhbypw0000gn/T/ipykernel_60431/1063128591.py:88: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  table['Logistic'][idex]= acc_logistic[number]
/var/folders/1s/70xy4dvs6mn0zqfz6nvhbypw0000gn/T/ipykernel_60431/1

In [17]:
result # p=2

,Logistic,SVM_linear,SVM_Nonlinear,KNN,Random,Decision,Navie
PCA,0.883333,0.875,0.908333,0.908333,0.916667,0.9,0.9


In [18]:
random_classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = random(x_train,y_train,x_test)
random_classifier

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)

In [ ]:
#### Accuracy score for the Random forest classfication model is high... so will save the model for deployment....


In [19]:
# Save the Random forest classfication model

pickle.dump(random_classifier,open("Finalized_Random_Model",'wb'))

In [21]:
pickle.dump(scX,open("SCX.pk",'wb'))